In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [4]:
df = pd.read_csv('eda_missing_data_dataset1.csv')
df.head()

,date,center_point_geom,longitude,latitude,number_of_strikes
0,2018-08-01,POINT(-81.6 22.6),-81.6,22.6,48
1,2018-08-01,POINT(-81.1 22.6),-81.1,22.6,32
2,2018-08-01,POINT(-80.9 22.6),-80.9,22.6,118
3,2018-08-01,POINT(-80.8 22.6),-80.8,22.6,69
4,2018-08-01,POINT(-98.4 22.8),-98.4,22.8,44


In [5]:
df.shape

(717530, 5)

In [6]:
df_zip = pd.read_csv('eda_missing_data_dataset2.csv')
df_zip.head()

,date,zip_code,city,state,state_code,center_point_geom,number_of_strikes
0,2018-08-08,3281,Weare,New Hampshire,NH,POINT(-71.7 43.1),1
1,2018-08-14,6488,Heritage Village CDP,Connecticut,CT,POINT(-73.2 41.5),3
2,2018-08-16,97759,"Sisters city, Black Butte Ranch CDP",Oregon,OR,POINT(-121.4 44.3),3
3,2018-08-18,6776,New Milford CDP,Connecticut,CT,POINT(-73.4 41.6),48
4,2018-08-08,1077,Southwick,Massachusetts,MA,POINT(-72.8 42),2


In [8]:
df_zip.shape

(323700, 7)

In [14]:
df_joined = df.merge(df_zip, how='left', on=['date' , 'center_point_geom'])
df_joined.head()


,date,center_point_geom,longitude,latitude,number_of_strikes_x,zip_code,city,state,state_code,number_of_strikes_y
0,2018-08-01,POINT(-81.6 22.6),-81.6,22.6,48,NaN,NaN,NaN,NaN,NaN
1,2018-08-01,POINT(-81.1 22.6),-81.1,22.6,32,NaN,NaN,NaN,NaN,NaN
2,2018-08-01,POINT(-80.9 22.6),-80.9,22.6,118,NaN,NaN,NaN,NaN,NaN
3,2018-08-01,POINT(-80.8 22.6),-80.8,22.6,69,NaN,NaN,NaN,NaN,NaN
4,2018-08-01,POINT(-98.4 22.8),-98.4,22.8,44,NaN,NaN,NaN,NaN,NaN


In [15]:
df_joined.describe()

,longitude,latitude,number_of_strikes_x,zip_code,number_of_strikes_y
count,717530.000000,717530.000000,717530.000000,323700.000000,323700.000000
mean,-90.875445,33.328572,21.637081,57931.958996,25.410587
std,13.648429,7.938831,48.029525,22277.327411,57.421824
min,-133.900000,16.600000,1.000000,1002.000000,1.000000
25%,-102.800000,26.900000,3.000000,38260.750000,3.000000
50%,-90.300000,33.200000,6.000000,59212.500000,8.000000
75%,-80.900000,39.400000,21.000000,78642.000000,24.000000
max,-43.800000,51.700000,2211.000000,99402.000000,2211.000000


In [18]:
missing_values = pd.isnull(df_joined.state_code)
missing_values.head()

0    True
1    True
2    True
3    True
4    True
Name: state_code, dtype: bool

In [19]:
df_null_geo = df_joined[missing_values]
df_null_geo.head()

,date,center_point_geom,longitude,latitude,number_of_strikes_x,zip_code,city,state,state_code,number_of_strikes_y
0,2018-08-01,POINT(-81.6 22.6),-81.6,22.6,48,NaN,NaN,NaN,NaN,NaN
1,2018-08-01,POINT(-81.1 22.6),-81.1,22.6,32,NaN,NaN,NaN,NaN,NaN
2,2018-08-01,POINT(-80.9 22.6),-80.9,22.6,118,NaN,NaN,NaN,NaN,NaN
3,2018-08-01,POINT(-80.8 22.6),-80.8,22.6,69,NaN,NaN,NaN,NaN,NaN
4,2018-08-01,POINT(-98.4 22.8),-98.4,22.8,44,NaN,NaN,NaN,NaN,NaN


In [20]:
df_null_geo.shape

(393830, 10)

In [40]:
top_missing = df_null_geo[['latitude' , 'longitude' , 'number_of_strikes_x']].groupby(['latitude' , 'longitude']).sum().sort_values('number_of_strikes_x' , ascending=False).reset_index()


In [41]:
top_missing.head(10)

,latitude,longitude,number_of_strikes_x
0,22.4,-84.2,3841
1,22.9,-82.9,3184
2,22.4,-84.3,2999
3,22.9,-83.0,2754
4,22.5,-84.1,2746
5,22.5,-84.2,2738
6,22.3,-81.0,2680
7,22.9,-82.4,2652
8,22.9,-82.3,2618
9,22.3,-84.3,2551


In [42]:
import plotly.express as px  # Be sure to import express
# reduce size of db otherwise it could break
fig = px.scatter_geo(top_missing[top_missing.number_of_strikes_x>=300],  # Input Pandas DataFrame
                    lat="latitude",  # DataFrame column with latitude
                    lon="longitude",  # DataFrame column with latitude
                    size="number_of_strikes_x") # Set to plot size as number of strikes
fig.update_layout(
    title_text = 'Missing data', # Create a Title
)

fig.show()

In [43]:
import plotly.express as px  # Be sure to import express
fig = px.scatter_geo(top_missing[top_missing.number_of_strikes_x>=300],  # Input Pandas DataFrame
                    lat="latitude",  # DataFrame column with latitude
                    lon="longitude",  # DataFrame column with latitude
                    size="number_of_strikes_x") # Set to plot size as number of strikes
fig.update_layout(
    title_text = 'Missing data', # Create a Title
    geo_scope='usa',  # Plot only the USA instead of globe
)

fig.show()

**Merge Function Explanation**

In [56]:
one_data = {
    'Planets': ['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn',  'Neptune'],
    'Distance from Sun': [57.9, 108.2, 149.6, 227.9, 778.6, 1433.5,  449],
    'Moons': [0, 0, 1, 2, 79, 82, 27]
}
df_1 = pd.DataFrame(one_data)
df_1

,Planets,Distance from Sun,Moons
0,Mercury,57.9,0
1,Venus,108.2,0
2,Earth,149.6,1
3,Mars,227.9,2
4,Jupiter,778.6,79
5,Saturn,1433.5,82
6,Neptune,449.0,27


In [59]:
two_Data = {
    'planets' : ['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter',  'Uranus', 'Neptune'],
    'Distance from Sun': [57.9, 108.2, 149.6, 227.9, 778.6, 1433,  449.6 ],
    'Moons': [0, 0, 1, 2, 79, 82, 27],
    'Planet Type': ['Terrestrial', 'Terrestrial', 'Terrestrial', 'Terrestrial', 'Gas Giant', 'Gas Giant',  'Ice Giant']
}
df_2 = pd.DataFrame(two_Data)
df_2


,planets,Distance from Sun,Moons,Planet Type
0,Mercury,57.9,0,Terrestrial
1,Venus,108.2,0,Terrestrial
2,Earth,149.6,1,Terrestrial
3,Mars,227.9,2,Terrestrial
4,Jupiter,778.6,79,Gas Giant
5,Uranus,1433.0,82,Gas Giant
6,Neptune,449.6,27,Ice Giant


In [62]:
merge_data = df_1.merge(df_2, how='left', on = ['Moons' , 'Distance from Sun' ])
merge_data

,Planets,Distance from Sun,Moons,planets,Planet Type
0,Mercury,57.9,0,Mercury,Terrestrial
1,Venus,108.2,0,Venus,Terrestrial
2,Earth,149.6,1,Earth,Terrestrial
3,Mars,227.9,2,Mars,Terrestrial
4,Jupiter,778.6,79,Jupiter,Gas Giant
5,Saturn,1433.5,82,NaN,NaN
6,Neptune,449.0,27,NaN,NaN
